## Amazon SageMaker to build, train, and deploy a machine learning (ML) model using the XGBoost ML algorithm
From aws Tutorial: https://aws.amazon.com/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/

### Steps
* Create a SageMaker notebook instance
* Prepare the data
* Train the model to learn from the data
* Deploy the model
* Evaluate your ML model's performance

### Step 1: Create an Amazon SageMaker notebook instance
* Sign in to the Amazon SageMaker console,  select your preferred AWS Region. 
* choose Notebook instances, then choose Create notebook instance.

### Step 2: Prepare the data

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Matplotlib is building the font cache; this may take a moment.


Success - the MySageMakerInstance is in the us-west-1 region. You will use the 632365934929.dkr.ecr.us-west-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [3]:
bucket_name = 'aws-sagemaker-ml-prac' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-eest-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [4]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [5]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [6]:
model_data.shape

(41188, 61)

In [7]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   age                            41188 non-null  int64
 1   campaign                       41188 non-null  int64
 2   pdays                          41188 non-null  int64
 3   previous                       41188 non-null  int64
 4   no_previous_contact            41188 non-null  int64
 5   not_working                    41188 non-null  int64
 6   job_admin.                     41188 non-null  int64
 7   job_blue-collar                41188 non-null  int64
 8   job_entrepreneur               41188 non-null  int64
 9   job_housemaid                  41188 non-null  int64
 10  job_management                 41188 non-null  int64
 11  job_retired                    41188 non-null  int64
 12  job_self-employed              41188 non-null  int64
 13  job_services    

In [8]:
model_data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.0,32.0,38.0,47.0,98.0
campaign,41188.0,2.567593,2.770014,1.0,1.0,2.0,3.0,56.0
pdays,41188.0,962.475454,186.910907,0.0,999.0,999.0,999.0,999.0
previous,41188.0,0.172963,0.494901,0.0,0.0,0.0,0.0,7.0
no_previous_contact,41188.0,0.963217,0.188230,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
poutcome_failure,41188.0,0.103234,0.304268,0.0,0.0,0.0,0.0,1.0
poutcome_nonexistent,41188.0,0.863431,0.343396,0.0,1.0,1.0,1.0,1.0
poutcome_success,41188.0,0.033335,0.179512,0.0,0.0,0.0,0.0,1.0
y_no,41188.0,0.887346,0.316173,0.0,1.0,1.0,1.0,1.0


In [16]:
model_data.isnull().sum()

age                     0
campaign                0
pdays                   0
previous                0
no_previous_contact     0
                       ..
poutcome_failure        0
poutcome_nonexistent    0
poutcome_success        0
y_no                    0
y_yes                   0
Length: 61, dtype: int64

In [15]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print("Train:", train_data.shape)
print("Test:", test_data.shape)

Train: (28831, 61)
Test: (12357, 61)


In [17]:
train_data.head(2)

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
40949,54,3,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
9332,56,2,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0


### Step 3: Train the ML model

In [18]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [19]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [20]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-04-28-01-30-39-149


2023-04-28 01:30:39 Starting - Starting the training job...
2023-04-28 01:30:55 Starting - Preparing the instances for training......
2023-04-28 01:31:58 Downloading - Downloading input data...
2023-04-28 01:32:28 Training - Downloading the training image.....
2023-04-28 01:33:13 Training - Training image download completed. Training in progress.Arguments: train
[2023-04-28:01:33:24:INFO] Running standalone xgboost training.
[2023-04-28:01:33:24:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-04-28:01:33:24:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8596.23mb
[2023-04-28:01:33:24:INFO] Determined delimiter of CSV input is ','
[01:33:24] S3DistributionType set as FullyReplicated
[01:33:24] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[01:33:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 14 pruned nodes, max_depth=5
[0]#011train

### Step 4: Deploy the model

In [21]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-04-28-01-35-51-538
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-04-28-01-35-51-538
INFO:sagemaker:Creating endpoint with name xgboost-2023-04-28-01-35-51-538


------!

In [22]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


### Step 5: Evaluate model performance

In [23]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



### Step 6: Clean up
a. Delete your endpoint: 
b. Delete your training artifacts and S3 bucket: 
c. Delete your SageMaker Notebook:


In [24]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2023-04-28-01-35-51-538
INFO:sagemaker:Deleting endpoint with name: xgboost-2023-04-28-01-35-51-538


In [25]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'BYFQQ2REWH2C8D2S',
   'HostId': 'jemv6Pjvw1olo6x2hRed+hjisOm0eoJZC6513aP1VRs04HW7T4d0B8cIe74ESwkGvtymploi0sg=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'jemv6Pjvw1olo6x2hRed+hjisOm0eoJZC6513aP1VRs04HW7T4d0B8cIe74ESwkGvtymploi0sg=',
    'x-amz-request-id': 'BYFQQ2REWH2C8D2S',
    'date': 'Fri, 28 Apr 2023 01:42:40 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-04-28-01-30-39-149/output/model.tar.gz'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-04-28-01-30-39-149/profiler-output/system/incremental/2023042801/1682645580.algo-1.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/train/train.csv'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-04-28-01-30-39-149/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/ou